In [30]:
import pandas as pd
import re
import numpy as np

In [96]:
naver = pd.read_csv('../asset/250327_한한중오픈프로.csv')
kcndic = pd.read_csv('../asset/250327_한한중사전센터.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_3444\1815217456.py:2: DtypeWarning: Columns (27,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  kcndic = pd.read_csv('../asset/250327_한한중사전센터.csv')


##### 데이터 전처리

In [34]:
# naver.fillna('', inplace=True)
# kcndic.fillna('', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_3444\3154040114.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  naver.fillna('', inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_3444\3154040114.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  kcndic.fillna('', inplace=True)


In [98]:
naver.replace(r'^\s*$', np.nan, regex=True, inplace=True)
kcndic.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [99]:
n_backup = naver.copy()
k_backup = kcndic.copy()


##### 엔트리ID 매칭들 중 안쓰게된거

In [ ]:
# def chunked_merge(naver, kcndic, chunk_size=50000):
#     # ✅ key 생성 (표제어 + 제공처ID)
#     naver['key'] = naver['표제어'] + '_' + naver['제공처ID'].astype(str)
#     kcndic['key'] = kcndic['표제어'] + '_' + kcndic['제공처ID'].astype(str)

#     # ✅ naver는 표제어 기준으로 중복 제거
#     naver_sub = naver[['key', '엔트리ID', '편집자', '마지막편집시간', '표제어종류', '부모표제어ID', '원어멤버ID', '발음ID', '별명ID(이형태부)', '오늘의단어선정여부', '서비스노출(표제어부)', '검색노출(표제어부)']].drop_duplicates('key')
    
#     # 🔄 B (kcndic)를 chunk 단위로 나눠서 병합
#     chunks = []
#     total = len(kcndic)
    
#     for i in range(0, total, chunk_size):
#         kcndic_chunk = kcndic.iloc[i:i + chunk_size].copy()
#         merged_chunk = pd.merge(kcndic_chunk, naver_sub, on='key', how='left', suffixes=('', '_N'))
        
#         # 덮어쓰기
#         for col in naver_sub.columns.difference(['key']):
#             merged_chunk[col] = merged_chunk[col + '_N']
        
#         # 불필요한 컬럼 제거
#         merged_chunk.drop(columns=[col + '_N' for col in naver_sub.columns.difference(['key'])], inplace=True)
        
#         chunks.append(merged_chunk)
#         print(f"✅ Chunk {i//chunk_size + 1} 병합 완료")

#     # 🔗 모든 chunk 합치기
#     merged = pd.concat(chunks, ignore_index=True)
#     print("🎉 전체 병합 완료!")
    
#     return merged

# result = chunked_merge(naver, kcndic)

# result.to_csv('../result/250324_result4_엔트리ID만_통합_청크별.csv', index=False)
# 250324: 한한중 전체 뽑아와서 표제어(제공처ID) 기준 정보옮겨오기 성공
# 다음에오면: 뜻풀이 기준 정보 옮겨오기

In [ ]:
# def chunked_merge_with_logs(naver, kcndic, chunk_size=50000):

#     # ✅ key 생성
#     naver['key'] = naver['표제어'] + '_' + naver['제공처ID'].astype(str)
#     kcndic['key'] = kcndic['표제어'] + '_' + kcndic['제공처ID'].astype(str)

#     # ✅ 1. 중복된 key 로그 저장
#     dup_keys = naver[naver.duplicated('key', keep=False)]
#     if not dup_keys.empty:
#         dup_keys.to_excel('../log/중복_key_naver.xlsx', index=False)
#         print(f"🟡 naver 중복 key {len(dup_keys)}건 저장됨")

#     # ✅ 2. 고유 key만 추출
#     naver_sub = naver[['key', '엔트리ID', '편집자', '마지막편집시간', '표제어종류', '부모표제어ID', '원어멤버ID', 
#                        '발음ID', '별명ID(이형태부)', '오늘의단어선정여부', '서비스노출(표제어부)', '검색노출(표제어부)']
#                      ].drop_duplicates('key')

#     chunks = []
#     unmatched_rows = []

#     total = len(kcndic)
#     for i in range(0, total, chunk_size):
#         kcndic_chunk = kcndic.iloc[i:i + chunk_size].copy()
#         merged_chunk = pd.merge(kcndic_chunk, naver_sub, on='key', how='left', suffixes=('', '_N'))

#         # 덮어쓰기
#         for col in naver_sub.columns.difference(['key']):
#             merged_chunk[col] = merged_chunk[col + '_N']
#         merged_chunk.drop(columns=[col + '_N' for col in naver_sub.columns.difference(['key'])], inplace=True)

#         # ✅ 병합 실패한 엔트리ID 없는 row 수집
#         unmatched = merged_chunk[merged_chunk['엔트리ID'].isna()]
#         if not unmatched.empty:
#             unmatched_rows.append(unmatched[['표제어', '제공처ID']].drop_duplicates())

#         chunks.append(merged_chunk)
#         print(f"✅ Chunk {i//chunk_size + 1} 병합 완료")

#     # 전체 병합
#     merged = pd.concat(chunks, ignore_index=True)

#     # ✅ 매칭 안된 kcndic 로그 저장
#     if unmatched_rows:
#         unmatched_all = pd.concat(unmatched_rows).drop_duplicates()
#         unmatched_all.to_excel('../log/엔트리ID_매칭실패_kcndic.xlsx', index=False)
#         print(f"🟥 매칭 실패 kcndic 표제어 {len(unmatched_all)}건 로그 저장됨")

#     print("🎉 전체 병합 완료!")
#     return merged


In [ ]:
# # 가장 빠르고 오류없는듯! 
# def safe_chunked_merge_with_logs2(naver, kcndic, chunk_size=50000):
#     ### 🔑 key 생성 (유효한 행만)
#     def make_key(df, source):
#         df['key'] = None
#         valid_mask = (
#             df['표제어'].notna() & (df['표제어'].astype(str).str.strip() != '') &
#             df['제공처ID'].notna() & (df['제공처ID'].astype(str).str.strip() != '')
#         )
#         # valid_mask = (df['표제어'] != '') & (df['제공처ID'] != '')
#         df.loc[valid_mask, 'key'] = df.loc[valid_mask, '표제어'] + '_' + df.loc[valid_mask, '제공처ID'].astype(str)

#         # 🟡 불완전 key 로그 저장 (표제어나 제공처ID 중 하나만 비어 있음)
#         partial_null_mask = (
#             ((df['표제어'].isna()) | (df['표제어'].astype(str).str.strip() == '')) &
#             ((df['제공처ID'].notna()) & (df['제공처ID'].astype(str).str.strip() != ''))
#         ) | (
#             ((df['표제어'].notna()) & (df['표제어'].astype(str).str.strip() != '')) &
#             ((df['제공처ID'].isna()) | (df['제공처ID'].astype(str).str.strip() == ''))
#         )
#         # partial_null_mask = (
#         #     (df['표제어'] == '') & (df['제공처ID'] != '')
#         # ) | (
#         #     (df['표제어'] != '') & (df['제공처ID'] == '')
#         # )
#         partial_nulls = df[partial_null_mask]
#         if not partial_nulls.empty:
#             partial_nulls.to_excel(f"../log/250000_result0_{source}_불완전_key_로그.xlsx", index=False)
#             print(f"🟠 {source} 내 불완전 key {len(partial_nulls)}건 로그 저장됨")

#         return df

#     # 1️⃣ 키 생성
#     naver = make_key(naver, 'naver')
#     kcndic = make_key(kcndic, 'kcndic')

#     # 2️⃣ 중복 key 로그 저장 (naver only)
#     dup_keys = naver[naver.duplicated('key', keep=False) & naver['key'].notna()]
#     if not dup_keys.empty:
#         dup_keys.to_excel('../log/250000_result0_중복_key_naver.xlsx', index=False)
#         print(f"🟡 naver 중복 key {len(dup_keys)}건 로그 저장됨")

#     # 3️⃣ 고유 key만 추출
#     headword_cols = [
#         '번호', '표제어번호', '엔트리ID', '집필상태', '편집자', 
#         '최초등록시간', '마지막편집시간', '표제어종류', 
#         '부모표제어ID', '원어멤버ID', 
#         '발음ID', '발음종류', '여성발음파일경로', 
#         '별명ID(이형태부)', '오늘의단어선정여부', '서비스노출(표제어부)', '검색노출(표제어부)'
#     ]
#     # naver_sub = naver[['key'] + headword_cols].drop_duplicates('key')
#     naver_sub = naver[naver['key'].notna()][['key'] + headword_cols].drop_duplicates('key')

    
#     # 4️⃣ chunk 병합
#     chunks = []
#     unmatched_rows = []
#     total = len(kcndic)

#     for i in range(0, total, chunk_size):
#         kc_chunk = kcndic.iloc[i:i+chunk_size].copy()

#         merged_chunk = pd.merge(kc_chunk, naver_sub, on='key', how='left', suffixes=('', '_N'))

#         for col in headword_cols:
#             merged_chunk[col] = merged_chunk[col + '_N']
        
#         merged_chunk.drop(columns=[col + '_N' for col in headword_cols], inplace=True)

#         # 매칭 실패 엔트리ID 로그
#         unmatched = merged_chunk[merged_chunk['엔트리ID'].isna() & merged_chunk['key'].notna()]
        
#         if not unmatched.empty:
#             unmatched_rows.append(unmatched[['표제어', '제공처ID']].drop_duplicates())

#         chunks.append(merged_chunk)
#         print(f"✅ Chunk {i//chunk_size + 1} 병합 완료")

#     # 5️⃣ 전체 병합 결과
#     merged = pd.concat(chunks, ignore_index=True)

#     # 6️⃣ 매칭 실패 로그 저장
#     if unmatched_rows:
#         unmatched_all = pd.concat(unmatched_rows).drop_duplicates()
#         unmatched_all.to_excel('../log/250000_result0_엔트리ID_매칭실패_kcndic.xlsx', index=False)
#         print(f"🟥 엔트리ID 매칭 실패 {len(unmatched_all)}건 로그 저장됨")

#     print("🎉 전체 병합 완료!")
#     return merged


In [80]:
# result2 = safe_chunked_merge_with_logs2(naver, kcndic)
# # 역시 가장 깔끔한듯? 3초라니..
 

🟠 naver 내 불완전 key 1311건 로그 저장됨
✅ Chunk 1 병합 완료
✅ Chunk 2 병합 완료
🟥 엔트리ID 매칭 실패 5044건 로그 저장됨
🎉 전체 병합 완료!


##### 엔트리ID 매칭

In [ ]:
# ㄱㄱ
def entryId_merge(naver, kcndic, chunk_size=50000):
    ### 🔑 key 생성 (유효한 행만)
    def make_key(df, source):
        df['key'] = None

        # 유효한 행: [표제어], [제공처ID] 모두 NaN아님
        valid_mask = df['표제어'].notna() & df['제공처ID'].notna()
        df.loc[valid_mask, 'key'] = df.loc[valid_mask, '표제어'] + '_' + df.loc[valid_mask, '제공처ID'].astype(str)

        # 🟡 불완전 key 로그 저장 (표제어나 제공처ID 중 하나만 비어 있음)
        invalid_mask = (
            df['표제어'].isna() & df['제공처ID'].notna() |
            df['표제어'].notna() & df['제공처ID'].isna()
        )
        invalid_rows = df[invalid_mask]
        if not invalid_rows.empty:
            # invalid_rows.to_excel(f"../log/250000_result0_{source}_불완전_key_로그.xlsx", index=False)
            print(f"🟠 {source} 내 매칭불가 key {len(invalid_rows)}건 로그 저장됨")
        return df

    # 1️⃣ 키 생성
    naver = make_key(naver, 'naver')
    kcndic = make_key(kcndic, 'kcndic')

    # 2️⃣ 중복 key 로그 저장 (naver only)
    dup_keys = naver[naver.duplicated('key', keep=False) & naver['key'].notna()]
    if not dup_keys.empty:
        # dup_keys.to_excel('../log/250000_result0_중복_key_naver.xlsx', index=False)
        print(f"🟡 naver 중복 key {len(dup_keys)}건 로그 저장됨")

    # 3️⃣ 고유 key만 추출
    headword_cols = [
        '번호', '표제어번호', '엔트리ID', '집필상태', '편집자', 
        '최초등록시간', '마지막편집시간', '표제어종류', 
        '부모표제어ID', '원어멤버ID', 
        '발음ID', '발음종류', '여성발음파일경로', 
        '별명ID(이형태부)', '종류(이형태부)', '오늘의단어선정여부', '서비스노출(표제어부)', '검색노출(표제어부)'
    ]
    naver_sub = naver[naver['key'].notna()][['key'] + headword_cols].drop_duplicates('key')

    # 4️⃣ chunk 병합
    chunks = []
    unmatched_rows = []
    total = len(kcndic)

    for i in range(0, total, chunk_size):
        kc_chunk = kcndic.iloc[i:i+chunk_size].copy()

        merged_chunk = pd.merge(kc_chunk, naver_sub, on='key', how='left', suffixes=('', '_N'))

        for col in headword_cols:
            merged_chunk[col] = merged_chunk[col + '_N']

        merged_chunk.drop(columns=[col + '_N' for col in headword_cols], inplace=True)

        # 매칭 실패 엔트리ID 로그 => kcndic 표제어 중 엔트리ID 못받은 것들들
        unmatched = merged_chunk[merged_chunk['엔트리ID'].isna() & merged_chunk['key'].notna()]
        
        if not unmatched.empty:
            unmatched_rows.append(unmatched[['표제어', '제공처ID']].drop_duplicates())

        chunks.append(merged_chunk)
        print(f"✅ Chunk {i//chunk_size + 1} 병합 완료")

    # 5️⃣ 전체 병합 결과
    merged = pd.concat(chunks, ignore_index=True)

    # 6️⃣ 매칭 실패 로그 저장
    if unmatched_rows:
        unmatched_all = pd.concat(unmatched_rows).drop_duplicates()
        # unmatched_all.to_excel('../log/250000_result0_엔트리ID_매칭실패_kcndic.xlsx', index=False)
        print(f"🟥 엔트리ID 매칭 실패 {len(unmatched_all)}건 로그 저장됨")

    print("🎉 전체 병합 완료!")
    return merged


In [ ]:
# ㄱㄱ
result = entryId_merge(naver, kcndic)
# 야호!! 깔끔하고 빠르다. 이거 채택ㄱㄱ 
# naver 1311건 => 1000개는 전임자가 제공처ID 안넣음. 300개는 삭제해야하는 데이터....?인건가? 
# kcndic 5044건 => 1000개는 naver에서 불완전 key라 못붙음. 4000개정도는 신규어라 naver에 애초에 없었음.

🟠 naver 내 매칭불가 key 1311건 로그 저장됨
✅ Chunk 1 병합 완료
✅ Chunk 2 병합 완료
🟥 엔트리ID 매칭 실패 5044건 로그 저장됨
🎉 전체 병합 완료!


In [ ]:
# ㄱㄱ 
r_backup = result.copy()

In [ ]:
# ㄱㄱ
def entryID_retry_origin(naver, result):
    """
    1차 병합 실패 중 제공처ID 없어서 실패한 애들을
    표제어 + 원어 기반으로 엔트리ID 재병합 (retry_key)

    ✅ 기존 엔트리ID 등 유지
    ✅ retry 성공한 행만 정확하게 result에 병합
    ✅ index 의존 ❌, retry_key 기반으로 안전하게 업데이트
    """

    # 🔑 병합 대상: naver 중 제공처ID 없어 key 못 만든 애들
    naver_failed = naver[naver['key'].isna() & naver['표제어'].notna()].copy()
    print(f"🟠 제공처ID 없어 병합 실패한 naver 행 수: {len(naver_failed)}")

    # retry_key 생성
    naver_failed['retry_key'] = naver_failed['표제어']
    with_origin = naver_failed['원어'].notna()
    naver_failed.loc[with_origin, 'retry_key'] = (
        naver_failed.loc[with_origin, '표제어'] + '_' + naver_failed.loc[with_origin, '원어'].astype(str)
    )

    # 🔁 병합 대상 추출
    headword_cols = [
        '번호', '표제어번호', '엔트리ID', '집필상태', '편집자', 
        '최초등록시간', '마지막편집시간', '표제어종류', 
        '부모표제어ID', '원어멤버ID', 
        '발음ID', '발음종류', '여성발음파일경로', 
        '별명ID(이형태부)', '종류(이형태부)', '오늘의단어선정여부', '서비스노출(표제어부)', '검색노출(표제어부)'
    ]

    before_dedup = len(naver_failed)

    # 🔍 중복 제외된 행 추출
    duplicated_keys = naver_failed[naver_failed['retry_key'].notna()]['retry_key']
    duplicated_only = naver_failed[naver_failed['retry_key'].isin(duplicated_keys[duplicated_keys.duplicated()])]
    print("🔍 중복된 retry_key로 인해 제외된 행들:")
    print(duplicated_only[['retry_key', '표제어', '원어', '엔트리ID']])  # 일부만 보기


    naver_sub = naver_failed[naver_failed['retry_key'].notna()][['retry_key'] + headword_cols].drop_duplicates('retry_key')
    after_dedup = len(naver_sub)
    print(f"🟡 retry_key 중복으로 제외된 행 수: {before_dedup - after_dedup}")

    # 📌 result 내 retry 대상 생성
    result = result.copy()  # 원본 보호
    result['retry_key'] = None
    with_origin = result['원어'].notna()
    result.loc[result['표제어'].notna(), 'retry_key'] = result['표제어']
    result.loc[with_origin, 'retry_key'] = (
        result.loc[with_origin, '표제어'] + '_' + result.loc[with_origin, '원어'].astype(str)
    )

    retry_target = result[
        result['key'].notna() & result['엔트리ID'].isna() & result['retry_key'].notna()
    ][['retry_key']].drop_duplicates()

    # 병합 시도
    matched = pd.merge(
        retry_target,
        naver_sub,
        on='retry_key',
        how='left'
    )

    # 성공 / 실패 분리
    matched_success = matched[matched['엔트리ID'].notna()].copy()
    unmatched = matched[matched['엔트리ID'].isna()].copy()

    # ✅ 덮어쓰기 (retry_key 기준 merge-update)
    for col in headword_cols:
        result = result.merge(
            matched_success[['retry_key', col]],
            on='retry_key',
            how='left',
            suffixes=('', '_new')
        )
        result[col] = result[col].combine_first(result[col + '_new'])
        result.drop(columns=[col + '_new'], inplace=True)

    # ✅ 로그 저장
    if not matched_success.empty:
        matched_success[['retry_key']].to_excel('../log/retry_matched.xlsx', index=False)
        print(f"🟢 retry 병합 성공: {len(matched_success)}건 → retry_matched.xlsx 저장됨")

    if not unmatched.empty:
        unmatched[['retry_key']].to_excel('../log/retry_unmatched.xlsx', index=False)
        print(f"🔴 retry 병합 실패: {len(unmatched)}건 → retry_unmatched.xlsx 저장됨")

    print(f"🔁 retry 병합 완료: 총 {len(matched_success)}건 엔트리ID 보강")
    return result


In [ ]:
# ㄱㄱ
result2 = entryID_retry_origin(naver, result)  # 제공처ID 없는 애들 retry
result2.to_csv('../result/250330_id_result1_엔트리ID_제공처ID없던애들시도.csv', index=False)

# 250328: 와 이방법이 되네.... ㅜㅜㅜ 코드 한번 점검해보자 문제 없는지........ 하이제야진짜 뜻풀이에 들어갈수 있음...!!! 
# 250330: 아래 cell에 프린트된, 중복된 retry_key로 인해 제외된 행들의경우? 
#       - 그냥 나중에 수동으로 넣자..ㅎㅎ.. 

🟠 제공처ID 없어 병합 실패한 naver 행 수: 1311
🔍 중복된 retry_key로 인해 제외된 행들:
           retry_key   표제어       원어                             엔트리ID
100847           -라며   -라며      NaN  ff24baf2032c4894a8df69019ed656b7
109780           또르르   또르르      NaN  3eb9bac315724438b2824602d89b621b
115798           또르르   또르르      NaN  ba7f86bcbe9f4f9f998f6b44b9c29091
116851           -라며   -라며      NaN  4bd4034fe8344d35a2180f86811a71c7
129749           -기다   -기다      NaN  dee2e31ce16f44fba7a5d0f3fa3fccda
130199           -기다   -기다      NaN  2b8fe3640f934fb5a7a6f7b26e65965e
135027  튀르키예_Türkiye  튀르키예  Türkiye  0a5d4fcb947245d3910274d91208def0
138066  튀르키예_Türkiye  튀르키예  Türkiye  8290805ce54b4fda88271fbda2209f87
🟡 retry_key 중복으로 제외된 행 수: 4
🟢 retry 병합 성공: 983건 → retry_matched.xlsx 저장됨
🔴 retry 병합 실패: 4043건 → retry_unmatched.xlsx 저장됨
🔁 retry 병합 완료: 총 983건 엔트리ID 보강


##### 뜻풀이ID

target : 기존

kcndic : 250327_id_result2.1_잘못된전처리수정

In [ ]:
1. 엔트리ID를 가져오지 못한 표제어는 당연히 그 외 ID도 존재하지 않을것이므로 제외 필요 (엔트리 매칭은 할만큼했으니 이제 없는건 쿨하게 빼버리자!) 
    엔트리ID에 종속되는 구조를? 어케 처리할지도 생각좀.. 
2. 표제어종류: 단어, 속담, 숙어 분리할지 정해야함
3. 사실은걍 뭐... 엔트리ID랑 뜻풀이(처음n글자한정) 병합키 해버리면 제일 편할텐데... 요몇년사이 뜻풀이 수정된것도 많을거같단말이야 하 이거 어케 매칭함;; 
4. 뜻풀이...... 에서 뭐 a href처리같은거 해야하는데? 이건 또 어케함? 산넘어산..
    naver: {"ko":"형용사 ‘<a href=\"#/entry/kozh/89a2b79ee9304fdf9ee2948b43650933\">서글프다</a>’의 활용형. 해체의 의문형, 감탄형 등으로 쓰인다."}
    kcndic: {"ko":"형용사 ‘서글프다&EID=172456;’의 활용형. 해체의 의문형, 감탄형 등으로 쓰인다."}
    => 전처리 필요. 특별히 어렵진 않을듯. 근데 엔트리ID 매칭 안된 애면 어캄? 
    => 전처리 어려울시 일단 서로 다른 부위는 빼버리고 키로 만들자.. 

5. 아무튼 내가 쓸수있는것:
    표제어쪽정보, 품사, 뜻풀이, 서술형뜻풀이, 

In [ ]:
# ㄱㄱㄱ
r2_backup = result2.copy()

In [ ]:
# ㄱㄱ
# naver, kcndic 뜻풀이 전처리..
def clean_naver_definition(text):
    if pd.isna(text):
        return text
    # return re.sub(r'<a [^>]*?>.*?</a>', '', text)
    return re.sub(r'<a href=\\"[^>]+\\"?>(.*?)</a>', r'\1', text)

def clean_kcndic_definition(text):
    if pd.isna(text):
        return text
    return re.sub(r'&EID=\d+;', '', text)

def safe_fill_entry_id(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    entry_filled = df['엔트리ID'].ffill()
    word_filled = df['표제어'].ffill()
    fill_mask = df['표제어'].isna() & df['엔트리ID'].isna()
    valid_headwords = df.loc[df['표제어'].notna() & df['엔트리ID'].notna(), ['표제어', '엔트리ID']]
    headword_to_entry = valid_headwords.drop_duplicates('표제어').set_index('표제어')['엔트리ID'].to_dict()
    fill_mask &= word_filled.map(headword_to_entry).notna()
    df['엔트리ID_f'] = df['엔트리ID']
    df.loc[fill_mask, '엔트리ID_f'] = entry_filled[fill_mask]
    return df

In [ ]:
# ㄱㄱ
# ✅ 안전하게 엔트리ID 채우기
naver = safe_fill_entry_id(naver)  
result2 = safe_fill_entry_id(result2)


# 🔍 로그로 저장
naver[['엔트리ID_f', '표제어', '표제어종류']].to_excel('../log/250331_naver_entry_fill_.xlsx', index=False)
result2[['엔트리ID_f', '표제어', '표제어종류']].to_excel('../log/250331_kcndic_entry_fill_.xlsx', index=False)

In [ ]:
# ㄴㄴ
def meanId_merge_by_entryID_meaning(naver_df, kcndic_df):
    """
    [표제어] 기준으로 엔트리ID가 매칭된 kcndic에 대해,
    [엔트리ID + 뜻풀이] 기반으로 뜻풀이ID 병합

    병합 기준:
    - kcndic의 '엔트리ID'와 '뜻풀이'가 모두 존재해야 병합 대상
    - naver에서 '엔트리ID'와 '뜻풀이'가 모두 있는 행으로 병합 key 생성
    - 병합 key: 엔트리ID_뜻풀이
    """

    # ✅ 뜻풀이 전처리 (표현 차이 정리)
    naver_df['서술형뜻풀이_c'] = naver_df['서술형뜻풀이'].apply(clean_naver_definition).str.strip()
    kcndic_df['서술형뜻풀이_c'] = kcndic_df['서술형뜻풀이'].apply(clean_kcndic_definition).str.strip()

    # 🔑 naver: 병합용 key 생성 (엔트리ID, 뜻풀이 모두 있는 행만)
    naver_valid = naver_df[naver_df['엔트리ID_f'].notna() & naver_df['서술형뜻풀이_c'].notna()].copy()
    naver_valid['key'] = naver_valid['엔트리ID_f'].astype(str) + '_' + naver_valid['서술형뜻풀이_c']

    meaning_cols = [
        '뜻풀이ID'
    ]
    naver_keymap = naver_valid[['key', '뜻풀이ID']].drop_duplicates('key')
    # 중복된 key 몇개 제거됐나 

    print(f"🧩 naver 병합 대상 key 수: {len(naver_keymap)}")

    # 🔑 kcndic: 병합 대상 추출 (엔트리ID와 뜻풀이 모두 있는 행만)
    kcndic_valid = kcndic_df[kcndic_df['엔트리ID_f'].notna() & kcndic_df['서술형뜻풀이_c'].notna()].copy()
    kcndic_valid['key'] = kcndic_valid['엔트리ID_f'].astype(str) + '_' + kcndic_valid['서술형뜻풀이_c']

    print(f"🧩 kcndic 병합 대상 행 수: {len(kcndic_valid)}")

    # 병합 키 교집합, 차집합 추출
    naver_keys = set(naver_keymap['key'])
    kcndic_keys = set(kcndic_valid['key'])

    intersection_keys = naver_keys & kcndic_keys
    naver_only_keys = naver_keys - kcndic_keys
    kcndic_only_keys = kcndic_keys - naver_keys

    print(f"🔍 병합 key 교집합 수: {len(intersection_keys)}")
    print(f"📉 naver에만 존재하는 key 수: {len(naver_only_keys)}")
    print(f"📉 kcndic에만 존재하는 key 수: {len(kcndic_only_keys)}")
    
    # 🔻 차집합 로그: naver only
    naver_unmatched = naver_valid[naver_valid['key'].isin(naver_only_keys)][['최초등록시간', '편집자', '엔트리ID_f', '표제어', '표제어종류', 'key']]
    naver_unmatched.to_excel('../log/250330_key_naver_only.xlsx', index=False)

    # 🔻 차집합 로그: kcndic only
    kcndic_unmatched = kcndic_valid[kcndic_valid['key'].isin(kcndic_only_keys)][['최초등록시간', '편집자', '엔트리ID_f', '표제어', '표제어종류', 'key']]
    kcndic_unmatched.to_excel('../log/250330_key_kcndic_only.xlsx', index=False)

    # print(kcndic_valid.loc[kcndic_valid['key'].notna(), ['뜻풀이ID']].head(10))
    # print(naver_keymap.loc[naver_keymap['key'].notna(), ['뜻풀이ID']].head(10))
    print(naver_keymap['뜻풀이ID'].isna().sum())

    # 병합
    merged = pd.merge(kcndic_valid, naver_keymap, on='key', how='left', suffixes=('', '_N'))
    print(f"✅ 병합 완료: 매칭된 뜻풀이ID 수: {merged['뜻풀이ID'].notna().sum()} / {len(merged)}")

    # # 원래 kcndic_df에 뜻풀이ID 업데이트
    # updated_df = kcndic_df.copy()
    # updated_df.loc[merged.index, '뜻풀이ID'] = merged['뜻풀이ID']
    
    # 🛠 key 기반 병합 결과 반영
    merge_dict = merged.set_index('key')['뜻풀이ID'].dropna().to_dict()
    updated_df = kcndic_df.copy()
    updated_df['key'] = updated_df['엔트리ID_f'].astype(str) + '_' + updated_df['서술형뜻풀이_c']
    updated_df['뜻풀이ID'] = updated_df['key'].map(merge_dict)
    print("🧩 merge_dict keys:", list(merge_dict.keys())[:5])
    print("🧩 updated_df keys:", list(updated_df['key'].dropna())[:5])

    return updated_df


In [ ]:
# ㄴㄴ
result3 = meanId_merge_by_entryID_meaning(naver, result2)
# 1. 분명 naver에만 존재하는 키 수에는 삭제단어..인데 삭제처리 안된색히들 의 뜻풀이어쩌구가 있을거란 말이지 이거 뜻풀이 시작하기 전 전처리에서 지우고싶으면 어떻게 하지? 
# => 아맞다 ㅁㅊ !!!!!!! 이거이거 내가 아직 ohw만 해서 속담숙어가 없구나??? 함부로 지우면 안되네! 속담숙어를 .. ... 가져와야겠다.. 으앙 ... 물론당연히 삭제단어도있음.. 
# 2. 엔트리ID flll 작업 제대로 됐는지 ㅈㄴ수상한데? 이거 검사해봐야돼;; 로그뽑자..흑 => 코드 바꾸니 제대로 되네! 시간은 좀 길어졌다만.. 
# 3. 그리고 대충보니 뜻이 수정된 녀석이 역시 있넹... 일단 100퍼일치인애들 매칭하고, 매칭 실패한놈들만 대상으로 뭐 앞 8글자기준하거나, 아님 일치율 80퍼 이런걸로 하거나.. 


🧩 naver 병합 대상 key 수: 43941
🧩 kcndic 병합 대상 행 수: 41596
🔍 병합 key 교집합 수: 39609
📉 naver에만 존재하는 key 수: 4332
📉 kcndic에만 존재하는 key 수: 1986
2239
✅ 병합 완료: 매칭된 뜻풀이ID 수: 0 / 41596
🧩 merge_dict keys: []
🧩 updated_df keys: ['df66ab58126047a1bcf2da7f73a8fb87_{"ko":"어떤 장소나 물건의 바깥 경계에 가까운 부분."}', '2eaf13776c404e12857be56595dcedc2_{"ko":"행위의 주체임을 나타내는 말."}', '2eaf13776c404e12857be56595dcedc2_{"ko":"상태 변화의 대상임을 나타내는 말."}', '2eaf13776c404e12857be56595dcedc2_{"ko":"상태나 성질의 대상임을 나타내는 말."}', '2eaf13776c404e12857be56595dcedc2_{"ko":"<grammar>(‘되다’, ‘아니다’와 함께 쓰여)</grammar> 바뀌거나 부정되는 대상임을 나타내는 말."}']


In [ ]:
# ㄱㄱ
def meanId_merge_by_entryID_meaning2(naver_df, kcndic_df):
    """
    [표제어] 기준으로 엔트리ID가 매칭된 kcndic에 대해,
    [엔트리ID + 뜻풀이] 기반으로 뜻풀이ID 병합

    병합 기준:
    - kcndic의 '엔트리ID'와 '뜻풀이'가 모두 존재해야 병합 대상
    - naver에서 '엔트리ID'와 '뜻풀이'가 모두 있는 행으로 병합 key 생성
    - 병합 key: 엔트리ID_뜻풀이
    """

    # ✅ 뜻풀이 전처리 (표현 차이 정리)
    naver_df['서술형뜻풀이_c'] = naver_df['서술형뜻풀이'].apply(clean_naver_definition).str.strip()
    kcndic_df['서술형뜻풀이_c'] = kcndic_df['서술형뜻풀이'].apply(clean_kcndic_definition).str.strip()

    def make_key(df, source):
        df['key_m'] = None
        # 유효한 행: [엔트리ID_f], [서술형뜻풀이_c] 모두 NaN아님
        valid_mask = df['엔트리ID_f'].notna() & df['서술형뜻풀이_c'].notna()
        df.loc[valid_mask, 'key_m'] = df.loc[valid_mask, '엔트리ID_f'].astype(str) + '_' + df.loc[valid_mask, '서술형뜻풀이_c']
        return df
    
    # 키 생성
    naver_df = make_key(naver_df, 'naver')
    kcndic_df = make_key(kcndic_df, 'kcndic')

    # 고유 key만 추출
    meaning_cols = [
        '뜻풀이ID'
    ]
    naver_sub = naver_df[naver_df['key_m'].notna()][['key_m'] + meaning_cols].drop_duplicates('key_m')

    # chunk 병합
    chunks = []
    unmatched_rows = []
    total = len(kcndic_df)
    chunk_size = 50000
    
    for i in range(0, total, chunk_size):
        print(i,"번째반복")
        kc_chunk = kcndic_df.iloc[i:i+chunk_size].copy()

        merged_chunk = pd.merge(kc_chunk, naver_sub, on='key_m', how='left', suffixes=('', '_N'))

        for col in meaning_cols:
            merged_chunk[col] = merged_chunk[col + '_N']

        merged_chunk.drop(columns=[col + '_N' for col in meaning_cols], inplace=True)

        chunks.append(merged_chunk)
        print(f"✅ Chunk {i//chunk_size + 1} 병합 완료")

    # 전체 병합 결과
    merged = pd.concat(chunks, ignore_index=True)

    return merged


In [ ]:
# ㄱㄱ
result3 = meanId_merge_by_entryID_meaning2(naver, result2)
# 역시 이방법이 제대로 붙는군! 로그는 다 날라갔지만....ㅎㅎㅠㅠ.... 하... 
# 보니까 kcndic쪽 서술형뜻풀이에 <grammar>이 없어서 매칭 안되는 경우가 많아보이는데 뭘까? 확인요망... 

# 암튼 이 코드로 똑같이 생긴거 다 처리하면, 처리된개수, 안된개수 카운트... 안된 행 번호? 정보만 따로 return하자. 그리고 이제 80퍼일치율 이런것들을 매칭해주자

# 그전에 속담숙어 처리부터 추가할까봐 ~.~ 
# 250330: 새코드로만듦
#   - 250324_한한중오픈프로, 250328_id_result1.2_엔트리ID_제공처ID없던애들시도
#     얘네 열기!! 



0 번째반복
✅ Chunk 1 병합 완료
50000 번째반복
✅ Chunk 2 병합 완료


In [237]:
result3.to_csv('../result/250330_id_result4_뜻풀이ID_병합.csv', index=False)

In [164]:
naver.loc[naver['key'].notna(), 'key'] 

0            -ㄴ대_57092.0
2         서글펐는데_441213.0
5          바꿔 말하면_1869.0
6          마는구나_440503.0
7            출산_332169.0
               ...      
137443     국립 공원_39172.0
137445       고적지_25214.0
137446      꼬들꼬들_54625.0
137447       규범성_43968.0
137448      공명정대_28742.0
Name: key, Length: 30248, dtype: object